In [ ]:
# Stock Price Predictiong Using Python & Machine Learning with LSTM model (Long Short Term Memory)
pip install pandas_datareader

In [ ]:
# import libraries
import math                                    # mathematical functions        https://docs.python.org/3/library/math.html
import pandas_datareader as web                # data reader for panda         https://pandas-datareader.readthedocs.io/en/latest/
import numpy as np                             # numerical data in python      https://numpy.org/doc/stable/user/absolute_beginners.html
import pandas as pd                            # data analysis toolkit         https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt                # plotting data in figures      https://matplotlib.org/2.0.2/users/pyplot_tutorial.html
from sklearn.preprocessing import MinMaxScaler # raw data utility functions    https://scikit-learn.org/stable/modules/preprocessing.html
from keras import metrics as metrics
import tensorflow as tf
from keras.models import Sequential            # input output sequence of data https://www.tensorflow.org/guide/keras/sequential_model
from datetime import datetime, timedelta       # for manipulating dates        https://docs.python.org/3/library/datetime.html
from keras.layers import Dense, LSTM, Dropout  # layers for neural network     https://keras.io/api/layers/
plt.style.use('fivethirtyeight')               # plot style                    https://matplotlib.org/3.1.0/gallery/style_sheets/fivethirtyeight.html

In [ ]:
# Stock attributes
stock = 'GOOGL'
source = 'yahoo'
today = datetime.now() # todays date (YYYY-MM-DD HH:MM:SS)
years = 20            # years back to start from when grabbing stocks, i.e 10 years = 2020-10 = 2010
startYear = today-timedelta(days=365*years)

# Get stock data using pandas data reader
df = web.DataReader(stock, data_source=source, start=startYear, end=today)

df

In [ ]:
# Get rows and columns from the dataframe
df.shape

In [ ]:
# Show closing price history
plt.figure(figsize=(16,8))
plt.title('Closing Price History, source: ' + source)
plt.plot(df['Close'])
plt.xlabel(stock + ' stock data', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)

In [ ]:
# Create new dataframe with 'Close' column
data_target = df.filter(['Close'])

# Covert dataframe into a numpy array with a 1 day delay
dataset = data_target.shift(-1, fill_value=0).values

# Get the number of rows to train the model on, using 80% of the data
training_data_len = math.ceil(len(dataset) * .8)

# Scale data between 0 and 1 to avoid the bias using normalization
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Create training dataset
# Create the scaled training dataset
train_data = scaled_data[0:training_data_len , :]

# Split data into x_train and y_train datasets
time_interval = int(len(dataset)*0.05) # amount of steps to train on, I.E the amount of data the LSTM uses per "step" ex: [0,1,2....60] if time_interval = 10 first LSTM step will be 0-9 and predict 10, second step woule be 1-10 and predict 11. =
x_train = []   # past x days, using previous example this would be 0-9
y_train = []   # predicted target value.              this would be predicted value for 10

# loop for the last x days
for i in range(time_interval, len(train_data)):
    x_train.append(train_data[i-time_interval:i, 0]) # append closingprice for i day
    y_train.append(train_data[i, 0])             # append predicted value
    
# Convert x_train and y_train to numpy arrays to use in LSTM model
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape data from 2D to 3D
# LSTM network needs 3D input (number of samples, timesteps and features)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
# Build LSTM model
model = Sequential()

# # Run first layer with train data as input, 250 nodes, drop 20% when done, only return last output
# model.add(LSTM(250, return_sequences=True, input_shape=(x_train.shape[1], 1)))
# model.add(Dropout(0.2))

# # Run a second layer with 250 nodes, drop 10% 
# model.add(LSTM(250, return_sequences=True))
# model.add(Dropout(0.1))

# Run a third layer with 50 nodes, drop 10% when done return whole output
model.add(LSTM(500, return_sequences=False))
model.add(Dropout(0.1))

# Run a fourth layer with 25 nodes and 10% dropout
model.add(Dense(25))
model.add(Dropout(0.1))

model.add(Dense(units = 1))


In [ ]:
# Method for calculating RMSE used for loss in model.compile
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
# Compile model
# Optimizer is to improve upon the loss function https://keras.io/api/optimizers/adam/
# loss function is used to measure how good the model did on training in terms of RMSE
# Metrics is used to evaluate the accuracy of predicitons (currently does not work)
model.compile(optimizer='adam', loss=root_mean_squared_error, metrics=[
        metrics.MeanSquaredError(name="MSE"),
        metrics.MeanAbsoluteError(name="MAE"),
        metrics.MeanSquaredLogarithmicError(name="MSLE")
    ]
)

In [ ]:
# Train model
# Batch size is the total number of training examples present in a single batch.
# Epoch is the number of iterations when an entire dataset is passed forward and backward through a neural network
model.fit(x_train, y_train, batch_size=48, epochs=10)

In [ ]:
# Create testing dataset
# Create a new array containing scaled values
test_data = scaled_data[training_data_len - time_interval: :]

# Create the data sets x_test and y_test
x_test = [] # past x days
y_test = dataset[training_data_len:, :] # predicted value

# loop for the last x days
for i in range(time_interval, len(test_data)):
    x_test.append(test_data[i-time_interval:i, 0]) # append closing price for i day
    
# Convert data into a numpy array to use in LSTM model
x_test = np.array(x_test)

# Reshape data from 2D to 3D
# LSTM network needs 3D input (number of samples, timesteps and features)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test.shape

In [ ]:
# Get model predicted unscaled closingprice value
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
# calculates the accuracy of our predictions against the actual values within a given percentile
def accuracy_calc(percentage):
    actual = y_test
    ok_accuracy = 0
    total = actual.size
    threshold = (100 - percentage) / 100
    lower = 1-threshold
    upper = 1+threshold

    for i in range(total):
        cur_act = actual[i]
        cur_pred = predictions[i]
        if cur_act >= cur_pred*lower and cur_act <= cur_pred*upper:
            ok_accuracy +=1
    acc = ok_accuracy/total
    print(f"Percentage of predicted values within {percentage}% margin of actual: {acc}")

In [ ]:
print("RMSE: ", root_mean_squared_error(y_test, predictions))
accuracy_calc(50)
accuracy_calc(60)
accuracy_calc(70)
accuracy_calc(80)
accuracy_calc(90)
accuracy_calc(95)
accuracy_calc(99)

# Evaluate model 
model.evaluate(x_test, y_test, batch_size=8)

In [ ]:
# Plot data
train = data_target[:training_data_len]
valid = data_target[training_data_len:]
valid['Predictions'] = predictions

# Visualize data
plt.figure(figsize=(16,8))
plt.title('Predictions VS Actual')
plt.xlabel(stock + ' stock data', fontsize=18)
plt.ylabel('Closing Price USD ($)', fontsize=18)
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
# Show all valid and predicted prices
valid